In [1]:
# coding: utf-8
import tensorflow as tf
import numpy as np

from keras.models import Model
from keras.layers import Embedding , Input , Dense, Flatten , Activation ,\
                        GRU, LSTM
from keras import optimizers
from keras.callbacks import EarlyStopping 

import pickle 

import os
os.environ['CUDA_VISIBLE_DEVICES']='1' #用0這顆比較不會跟別人打架
os.environ['TF_FORCE_GPU_ALLOW_GROWTH']='true'
import tensorflow as tf
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.allow_soft_placement=True
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        print(e)
else:
    print('No GPU!!')
"""

@author: charlie
"""


[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 18232472681445650613
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 18341764441924178988
physical_device_desc: "device: XLA_CPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 10503562855
locality {
  bus_id: 1
  links {
  }
}
incarnation: 16091096537420782975
physical_device_desc: "device: 0, name: GeForce RTX 2080 Ti, pci bus id: 0000:02:00.0, compute capability: 7.5"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 12062879534506948838
physical_device_desc: "device: XLA_GPU device"
]
1 Physical GPUs, 1 Logical GPUs


Using TensorFlow backend.


'\n\n@author: charlie\n'

In [18]:
qlen , alen = 20 ,10


In [50]:
x,y=[],[]
post  , response = [] , []


with open('qa_dataset/stc_weibo_train_post', 'r' ) as f:
    for i in f.readlines():
        post.append(i.split())
        
with open('qa_dataset/stc_weibo_train_response', 'r' ) as f:
    for i in f.readlines():
        response.append(i.split())

for i,po in enumerate(post):
    if len(x)>=100000:
        break
    tmp_polen  , tmp_rsplen = len(po) , len(response[i])
    if tmp_polen>=alen  or tmp_rsplen>=alen or tmp_polen<3 or tmp_rsplen <3:
        continue
    x.append(po)
    y.append(response[i])

with open('post-response_10thousand_3_9.pkl', 'wb') as f:
    pickle.dump([x , y] , f)


In [51]:
with open('post-response_10thousand_3_9.pkl' , 'rb' ) as f:
    post, response  = pickle.load(f)

In [54]:
len(post) , len(response)

(100000, 100000)

In [55]:

class Tokenizer:
    
    def __init__(self , all_train_list, qlen =20, alen = 10):
        """traininig data only
        input list: post + response
        """
        self.qlen = qlen
        self.alen = alen
        
        self.word_uniq = {'EOS','BOS','OTHER'}
        self.word2id = dict()
        self.id2word = dict()
        
        for line in all_train_list:
            if len(line)==0:
                continue
                
            self.word_uniq |= set(line)
        self.word_uniq = sorted(list(self.word_uniq))
        self.num_word = len(self.word_uniq)
        for i,word in enumerate(self.word_uniq):
            self.word2id[word] =i+1
        
        self.word2id['PAD']=0
        self.PAD= self.word2id['PAD']
        self.OTHER=self.word2id['OTHER']
        self.EOS = self.word2id['EOS']
        self.BOS = self.word2id['BOS']
        
        for k,v in self.word2id.items():
            self.id2word[v]=k
        
    
    def token(self,  sents, source = 'q'):
        """ tokenize & padding 
        input/output: list of sentence
        """
        
        seq_len = 20 if source=='q' else 10
        
        tokened = []
        for line in sents:
            tmp = []
            for word in line:
                tmp.append(self.word2id.get(word , self.OTHER))
            tmp.append(self.EOS)
            tmp = tmp if len(tmp)<= seq_len else tmp[:seq_len-1]+[self.EOS]
            while len(tmp) <seq_len:
                tmp.append(self.PAD)
            tokened.append(tmp)
        return tokened
    def detoken(self, sents, space_unused_token=False):
        """ de-tokenize 
        input/output: list of sentence
        """
        detokened = []
        for line in sents:
            tmp = []
            for token in line:
                if space_unused_token and token in {self.OTHER,
                                                    self.EOS,
                                                    self.BOS,
                                                    self.PAD }:
                    continue
                tmp.append(self.id2word.get(token , ''))
            detokened.append(tmp)
        return detokened
        



In [56]:

post_train , post_test = post[:50000] , post[50000:]
response_train , response_test = response[:50000] , response[50000:]

tokenizer = Tokenizer(post_train+response_train )

In [57]:
print(f'num_word: {tokenizer.num_word}')

num_word: 52291


In [64]:
x , y = [],[]
x_test , y_test = [] , []

for line  in tokenizer.token(post_train):
    x.append(line)
    
for line in tokenizer.token(response_train):
    y.append(line)

for line  in tokenizer.token(post_test):
    x_test.append(line)
    
for line in tokenizer.token(response_test):
    y_test.append(line)

# x1,x2,y = np.array(x1) , np.array(x2) , np.array(y).reshape(-1 ,seq_len , 1)
# x1_train , x1_test = x1[:50000 ] , x1[50000: ]
# x2_train , x2_test = x2[:50000] , x2[50000:]
# y_train ,y_test = y[:50000] , y[50000:]

In [63]:
x[:3]

[[6524,
  38985,
  43345,
  43951,
  13559,
  26485,
  2789,
  1443,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [8998,
  44406,
  46834,
  37370,
  16082,
  37294,
  31326,
  14704,
  52078,
  1443,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [8127,
  41819,
  30312,
  23030,
  37294,
  50113,
  35555,
  34369,
  46912,
  1443,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0]]

In [62]:
for line in tokenizer.detoken(x[:3]):
    print(line)

['中国', '移动', '营销', '行来', '发展', '报告', 'alink', 'EOS', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD']
['你', '见', '过', '皮卡丘', '喝水', '的', '样子', '吗', '？', 'EOS', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD']
['从', '胚胎', '期', '开始', '的', '面部', '特征', '演变', '过程', 'EOS', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD']


In [ ]:


def get_model(nword = tokenizer.num_word , nunit=128 , embedding_dim=128 ):

    # declare
    input_x       =Input(shape=(tokenizer.qlen ,))
    input_teacher = Input(shape=(tokenizer.alen , ))
    embedding     = Embedding(nword+1, embedding_dim)    # token start from 1 and including PAD
    encoder = LSTM(nunit,return_state=True) 
    decoder = LSTM(nunit,return_sequences = True)
    lin = Dense(nword+1,activation='softmax')
    
    
    # inference
    emb_x       = embedding(input_x)
    emb_teacher = embedding(input_teacher)
    latent_enc ,h,c  = encoder(emb_x)
    latent_dec = decoder(emb_teacher , initial_state=[h,c])
    prob = lin(latent_dec)
    
    model = Model(inputs=[input_x, input_teacher] , outputs = prob)
    model.summary()
    
    return model

earlystop = EarlyStopping(monitor='loss',patience = 2 , verbose=2, mode='auto')
model = get_model()
model.compile(loss = 'sparse_categorical_crossentropy',
             optimizer='adam',
             metrics=['acc'])
    

